In [ ]:
#task1

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
attendance_raw =[{
    "student_id" : f"S{i:03}" ,
    "cohort" : np.random.choice(["alpha","beta","gamma"]),
    "attended_sessions" : np.random.randint(0,7),
    "expected_sessions" : 6
                }
for i in range(1,25)]


attendance = pd.DataFrame(attendance_raw)

print(attendance[:5])
attendance.info()

In [ ]:
#task2

In [ ]:
attendance_indexed = attendance.set_index("student_id")


In [ ]:
excused_absences = pd.Series(
    [1, 2, 1, 5, 2, 1, 3, 1, 2, 1],
    index=["S001", "S002", "S004", "S006", "S010",
           "S015", "S020", "S025", "S030", "S999"]
)

In [ ]:
attendance_indexed["adjusted_attendance"] = attendance_indexed["attended_sessions"] + excused_absences

In [ ]:
attendance_indexed["adjusted_attendance"] = attendance_indexed["adjusted_attendance"].fillna(attendance_indexed["attended_sessions"])

In [ ]:
attendance_indexed

In [ ]:
#task3

In [ ]:
attendance_indexed.iloc[0, 0] = "Beta  "
attendance_indexed.iloc[1, 0] = "  ALPHA "
attendance_indexed['cohort'] = attendance_indexed['cohort'].str.strip().str.lower()


In [ ]:
print(f"Unique cohorts: {attendance_indexed['cohort'].unique()}")

In [ ]:
#task4

In [ ]:
low_attendance = attendance_indexed[
    attendance_indexed['attended_sessions'] < attendance_indexed['expected_sessions']
].copy()

cohort_summary = attendance_indexed.groupby('cohort')['attended_sessions'].mean()


In [ ]:
print("Average Attendance by Cohort:")
print(cohort_summary)

In [ ]:
#task5

In [ ]:
attendance_indexed['attendance_ok'] = attendance_indexed['attended_sessions'] >= attendance_indexed['expected_sessions']
validation_check = low_attendance.index.isin(attendance_indexed[attendance_indexed['attendance_ok'] == True].index).any()


In [ ]:
if not validation_check:
    print("Validation passed")
else:
    print("Validation failed")